In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import *
import pickle

In [2]:
data = pickle.load(open("/Users/Maximus/downloads/Data_12_03", "rb"))

In [3]:
train_data = data.loc[data.Date < '2009-01-01', :]
dev_data = data.loc[(data.Date >= '2009-01-01') & (data.Date <= '2010-12-31'), :]
test_data = data.loc[data.Date >= '2011-01-01', :]

train_label = train_data.label
dev_label = dev_data.label
test_label = test_data.label

#train_text_features.shape
train_data['Surprise'].fillna(0, inplace=True)
dev_data['Surprise'].fillna(0, inplace=True)
#train_data.fillna(0, inplace=True)

/Users/Maximus/anaconda/envs/tensorflow/lib/python2.7/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
train_label.head()

13      UP
14      UP
15    DOWN
16      UP
17      UP
Name: label, dtype: object

In [56]:
train_data['label2'] = train_data['label'].apply(lambda x: 1 if str(x) == 'UP' else 0)

/Users/Maximus/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [57]:
train_data['label2'].shape

(17500,)

In [4]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')
train_text_features = vectorizer.fit_transform(train_data.bow)

dev_text_features = vectorizer.transform(dev_data.bow)

In [5]:
all_data_train = np.append(train_text_features.toarray(),\
                           np.array(train_data.Surprise).reshape(len(train_data.Surprise),1), 1)
all_data_dev = np.append(dev_text_features.toarray(),\
                         np.array(dev_data.Surprise).reshape(len(dev_data.Surprise),1), 1)

In [43]:
tf.reset_default_graph()  # clear away any leftovers from previous executions

input_dim = all_data_train.shape[1]
#train_label

X_ = tf.placeholder(tf.float32, shape=[None, input_dim], name="X")
y_ = tf.placeholder(tf.float32, shape=[None], name="y")

# Variables for the parameters of our model
with tf.name_scope('model_parameters'):
    w_ = tf.Variable(tf.zeros([input_dim, 1], dtype=tf.float32), name="w")
    b_ = tf.Variable(0.0, dtype=tf.float32, name="b")

# Output layer: \sigma(Xw + b)
with tf.name_scope('logit'):
    logits_ = tf.squeeze(tf.matmul(X_, w_)) + b_
with tf.name_scope('pred_proba'):
    pred_proba_ = tf.sigmoid(logits_)

# Cost function
with tf.name_scope('cost_function'):
    loss_ = tf.nn.sigmoid_cross_entropy_with_logits(logits_, y_, name='loss')

In [44]:
# Gradient descent training
with tf.name_scope('training'):
    alpha_ = tf.placeholder(tf.float32, name="learning_rate")
    optimizer_ = tf.train.GradientDescentOptimizer(alpha_)
    train_step_ = optimizer_.minimize(loss_)

# Initializer step
init_ = tf.initialize_all_variables()

In [61]:
max_steps = 100
print_every = max_steps / 10
alpha = 0.01  # learning rate

# We'll constrain this to run on CPU, as GPUs aren't very efficient
# on a model this small.
session = tf.Session(config=tf.ConfigProto(device_filters="/cpu:0"))
session.run(init_)  # initialize variables for this session


for i in xrange(max_steps):          
    # Run a single gradient descent step
    c, p, _ = session.run([loss_, pred_proba_, train_step_],
                           feed_dict={X_: all_data_train, y_: train_data.label2, alpha_: alpha})
        
    if (i % print_every == 0):
        avg_cost = sum(c) / len(train_data.label2)
        print "[iter %d] Average cost: %.03f" % (i, avg_cost)
        

# Run model over training set one last time
costs, y_pred = session.run([loss_, pred_proba_], 
                            feed_dict={X_: all_data_train, y_: train_data.label2})

print ""
print "Accuracy: %.03f" % np.mean(train_data.label2 == (y_pred >= 0.5))
print "Mean cross-entropy loss: %.03f" % np.mean(costs)


[iter 0] Average cost: 0.693
[iter 10] Average cost: 19639.211
[iter 20] Average cost: 2735.646
[iter 30] Average cost: 21331.093
[iter 40] Average cost: 2645.974
[iter 50] Average cost: 13573.046
[iter 60] Average cost: 2700.362
[iter 70] Average cost: 15810.444
[iter 80] Average cost: 3316.338
[iter 90] Average cost: 21808.962

Accuracy: 0.624
Mean cross-entropy loss: 2739.421


In [ ]:
fig = plt.figure()
plot_model(X, y, lambda X: session.run(pred_proba_, feed_dict={X_:X}))
plt.title("%d steps (final)" % max_steps)

In [7]:
import convol_net as CNN
CNN.test()

nano
